# 如何管理对话历史记录

持久性最常见的用例之一是用它来跟踪对话历史记录。这太棒了——它让继续对话变得容易。然而，随着对话变得越来越长，此对话历史记录可能会累积并占用越来越多的上下文窗口。这通常是不可取的，因为它会导致对 LLM 的调用更加昂贵和耗时，并且可能会出现错误。为了防止这种情况发生，您可能需要管理对话历史记录。

注意：本指南重点介绍如何在 LangGraph 中执行此操作，您可以在其中完全自定义执行此操作的方式。如果您想要更现成的解决方案，您可以查看 LangChain 中提供的功能：

- [如何过滤消息](https://js.langchain.com/docs/how_to/filter_messages/)
- [如何修剪消息](https://js.langchain.com/docs/how_to/trim_messages/)

## 设置

首先，让我们设置我们要使用的包
```bash
yarn add langchain @langchain/anthropic @langchain/core
```
接下来，我们需要为 Anthropic 设置 API 密钥（我们将使用的 LLM）
```bash
export ANTHROPIC_API_KEY=your_api_key
```

或者，我们可以为 [LangSmith 追踪](https://smith.langchain.com/) 设置 API 密钥，这将为我们提供一流的可观察性。
```bash
export LANGCHAIN_TRACING_V2="true"
export LANGCHAIN_CALLBACKS_BACKGROUND="true"
export LANGCHAIN_API_KEY=your_api_key
```

## 构建代理
现在让我们构建一个简单的 ReAct 风格代理。

In [2]:
import { ChatAnthropic } from "@langchain/anthropic";
import { tool } from "@langchain/core/tools";
import { BaseMessage, AIMessage } from "@langchain/core/messages";
import { StateGraph, Annotation, START, END } from "@langchain/langgraph";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { MemorySaver } from "@langchain/langgraph";
import { z } from "zod";

const AgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

const memory = new MemorySaver();

const searchTool = tool((_): string => {
    // 这是实际实现的占位符
    // 不过不要让LLM知道这一点😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
}, {
    name: "search",
    description: "Call to surf the web.",
    schema: z.object({
        query: z.string()
    })
})


const tools = [searchTool]
const toolNode = new ToolNode<typeof AgentState.State>(tools)
const model = new ChatAnthropic({ model: "claude-3-haiku-20240307" })
const boundModel = model.bindTools(tools)

function shouldContinue(state: typeof AgentState.State): "action" | typeof END {
  const lastMessage = state.messages[state.messages.length - 1];
  // 如果没有函数调用，那么我们就完成了
  if (lastMessage && !(lastMessage as AIMessage).tool_calls?.length) {
      return END;
  }
  // 否则，如果有，我们继续
  return "action";
}

// 定义调用模型的函数
async function callModel(state: typeof AgentState.State) {
  const response = await model.invoke(state.messages);
  // 我们返回一个对象，因为这将与现有状态合并
  return { messages: [response] };
}

// 定义一个新图
const workflow = new StateGraph(AgentState)
    // 定义我们将在其之间循环的两个节点
    .addNode("agent", callModel)
    .addNode("action", toolNode)
    // 我们现在添加一个条件边
    .addConditionalEdges(
        // 首先，我们定义起始节点。我们使用“代理”。
        // 这意味着这些是调用“agent”节点后获取的边。
        "agent",
        // 接下来，我们传入将确定接下来调用哪个节点的函数。
        shouldContinue
    )
    // 我们现在添加从“action”到“agent”的正常边缘。
    // 这意味着在调用“action”之后，接下来调用“agent”节点。
    .addEdge("action", "agent")
    // 将入口点设置为“agent”
    // 这意味着该节点是第一个被调用的节点
    .addEdge(START, "agent");

// 最后，我们编译它！
// 这会将其编译成 LangChain Runnable，
// 这意味着您可以像使用任何其他可运行程序一样使用它
const app = workflow.compile({
    checkpointer: memory,
});

In [3]:
import { HumanMessage } from "@langchain/core/messages";

const config = { configurable: { thread_id: "2"}, streamMode: "values" as const }

const inputMessage = new HumanMessage("hi! I'm bob");
for await (const event of await app.stream({
    messages: [inputMessage]
}, config)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (1) =================================`)
    console.log(recentMsg.content);
}

console.log("\n\n================================= END =================================\n\n")

const inputMessage2 = new HumanMessage("what's my name?");
for await (const event of await app.stream({
    messages: [inputMessage2]
}, config)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (2) =================================`)
    console.log(recentMsg.content);
}


================================ human Message (1) =================================
hi! I'm bob
================================ ai Message (1) =================================
Hello Bob! It's nice to meet you. I'm an AI assistant created by Anthropic. I'm here to help with any questions or tasks you may have. Please let me know if there's anything I can assist you with.


================================= END =================================


================================ human Message (2) =================================
what's my name?
================================ ai Message (2) =================================
Your name is Bob, as you introduced yourself earlier.


## 过滤消息

防止对话历史记录爆炸的最直接的方法是在消息传递给法学硕士之前过滤消息列表。这涉及两个部分：定义一个函数来过滤消息，然后将其添加到图中。请参阅下面的示例，它定义了一个非常简单的 `filterMessages` 函数，然后使用它。

In [4]:
import { ChatAnthropic } from "@langchain/anthropic";
import { tool } from "@langchain/core/tools";
import { BaseMessage, AIMessage } from "@langchain/core/messages";
import { StateGraph, Annotation, START, END } from "@langchain/langgraph";
import { ToolNode } from "@langchain/langgraph/prebuilt";
import { MemorySaver } from "@langchain/langgraph";
import { z } from "zod";

const MessageFilteringAgentState = Annotation.Root({
  messages: Annotation<BaseMessage[]>({
    reducer: (x, y) => x.concat(y),
  }),
});

const messageFilteringMemory = new MemorySaver();

const messageFilteringSearchTool = tool((_): string => {
    // 这是实际实现的占位符
    // 不过不要让LLM知道这一点😊
    return "It's sunny in San Francisco, but you better look out if you're a Gemini 😈."
}, {
    name: "search",
    description: "Call to surf the web.",
    schema: z.object({
        query: z.string()
    })
})

// 我们可以重复使用与上面相同的搜索工具，因为我们不需要在此示例中对其进行更改。
const messageFilteringTools = [messageFilteringSearchTool]
const messageFilteringToolNode = new ToolNode<typeof MessageFilteringAgentState.State>(messageFilteringTools)
const messageFilteringModel = new ChatAnthropic({ model: "claude-3-haiku-20240307" })
const boundMessageFilteringModel = messageFilteringModel.bindTools(messageFilteringTools)


async function shouldContinueMessageFiltering(state: typeof MessageFilteringAgentState.State): Promise<"action" | typeof END> {
    const lastMessage = state.messages[state.messages.length - 1];
    // 如果没有函数调用，那么我们就完成了
    if (lastMessage && !(lastMessage as AIMessage).tool_calls?.length) {
        return END;
    }
    // 否则，如果有，我们继续
    return "action";
}

const filterMessages = (messages: BaseMessage[]): BaseMessage[] => {
  // 这是非常简单的辅助函数，仅使用最后一条消息
  return messages.slice(-1);
}

// 定义调用模型的函数
async function callModelMessageFiltering(state: typeof MessageFilteringAgentState.State) {
  const response = await boundMessageFilteringModel.invoke(filterMessages(state.messages));
  // 我们返回一个对象，因为这将与现有状态合并
  return { messages: [response] };
}


// 定义一个新图
const messageFilteringWorkflow = new StateGraph(MessageFilteringAgentState)
  // 定义我们将在其之间循环的两个节点
  .addNode("agent", callModelMessageFiltering)
  .addNode("action", messageFilteringToolNode)
  // 我们现在添加一个条件边
  .addConditionalEdges(
    // 首先，我们定义起始节点。我们使用“代理”。
    // 这意味着这些是调用“agent”节点后获取的边。
    "agent",
    // 接下来，我们传入将确定接下来调用哪个节点的函数。
    shouldContinueMessageFiltering
  )
  // 我们现在添加从“action”到“agent”的正常边缘。
  // 这意味着在调用“action”之后，接下来调用“agent”节点。
  .addEdge("action", "agent")
  // 将入口点设置为“agent”
  // 这意味着该节点是第一个被调用的节点
  .addEdge(START, "agent");

// 最后，我们编译它！
// 这会将其编译成 LangChain Runnable，
// 这意味着您可以像使用任何其他可运行程序一样使用它
const messageFilteringApp = messageFilteringWorkflow.compile({
    checkpointer: messageFilteringMemory,
});

In [5]:
import { HumanMessage } from "@langchain/core/messages";

const messageFilteringConfig = { configurable: { thread_id: "2"}, streamMode: "values" as const }

const messageFilteringInput = new HumanMessage("hi! I'm bob");
for await (const event of await messageFilteringApp.stream({
    messages: [messageFilteringInput]
}, messageFilteringConfig)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (1) =================================`)
    console.log(recentMsg.content);
}

console.log("\n\n================================= END =================================\n\n")

const messageFilteringInput2 = new HumanMessage("what's my name?");
for await (const event of await messageFilteringApp.stream(
  {
    messages: [messageFilteringInput2]
  },
  messageFilteringConfig
)) {
    const recentMsg = event.messages[event.messages.length - 1];
    console.log(`================================ ${recentMsg._getType()} Message (2) =================================`)
    console.log(recentMsg.content);
}

================================ human Message (1) =================================
hi! I'm bob
================================ ai Message (1) =================================
Hello, nice to meet you Bob! I'm an AI assistant here to help out. Feel free to let me know if you have any questions or if there's anything I can assist with.


================================= END =================================


================================ human Message (2) =================================
what's my name?
================================ ai Message (2) =================================
I'm afraid I don't actually know your name, since you haven't provided that information to me. As an AI assistant, I don't have access to personal details about you unless you share them with me directly. I'm happy to continue our conversation, but I don't have enough context to know your specific name. Please feel free to introduce yourself if you'd like me to address you by name.


在上面的例子中，我们自己定义了 `filter_messages` 函数。我们还提供现成的方法来修剪和过滤 LangChain 中的消息。

- [如何过滤消息](https://js.langchain.com/docs/how_to/filter_messages/)
- [如何修剪消息](https://js.langchain.com/docs/how_to/trim_messages/)